In [1]:
import pandas as pd
import numpy as np
from utils.matrix_generator import generate_hilbert_matrix

In [2]:
generate_hilbert_matrix(
    n=3,
)

(array([[1.        , 0.5       , 0.33333333],
        [0.5       , 0.33333333, 0.25      ],
        [0.33333333, 0.25      , 0.2       ]]),
 array([1.83333333, 1.08333333, 0.78333333]))